In [ ]:
#!pip install polars

In [ ]:
#!pip install joblib

In [1]:
import pandas as pd
import re
from tqdm import tqdm
import polars as pl
from joblib import Parallel, delayed
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the second dataset (prescriptions)
df_prescriptions = pd.read_csv('/content/drive/My Drive/Drug Interaction/Drugs List - IHIO/source.csv')

In [17]:
path = '/content/drive/My Drive/Drug Interaction/Drugs List - IHIO'
print(os.listdir(path))

['source.csv', '_drug_interaction_filtered_df_final.csv', 'drug_interaction_sorted_without_duplicate_11.6M_final.csv', 'drug_interaction_sorted_without_duplicate_at_least_two_drugs_11.4M_final.csv', 'drug_interaction_sorted_without_duplicate_at_least_two_drugs_11.4M_final_with_drug_counts.csv']


In [3]:
df_bimarestani = pd.read_csv('/content/drive/My Drive/Drug Interaction/Drugs List - IHIO/bimarestani.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Drug Interaction/Drugs List - IHIO/bimarestani.csv'

In [ ]:
import pandas as pd
from collections import defaultdict

# Load the values to search from csv1
csv1 = pd.read_csv('/content/drive/My Drive/Drug Interaction/Drugs List - IHIO/bimarestani.csv')
csv1_column = 'generic code'

# Store values in a set for fast lookup
values_to_search = set(csv1[csv1_column].dropna().unique())

# Initialize a counter dictionary
counts = defaultdict(int)

# Read csv2 in chunks
csv2_column = 'pres_gen_code'
chunksize = 100_000  # adjust based on your RAM

for chunk in pd.read_csv('/content/drive/My Drive/Drug Interaction/Drugs List - IHIO/source.csv', usecols=[csv2_column], chunksize=chunksize):
    for value in chunk[csv2_column]:
        if value in values_to_search:
            counts[value] += 1

# Map counts back to csv1
csv1['count_of_bimarestani'] = csv1[csv1_column].map(lambda x: counts.get(x, 0))

# Show or save result
print(csv1)
# csv1.to_csv('csv1_with_counts.csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Drug Interaction/Drugs List - IHIO/bimarestani.csv'

In [ ]:
df_prescriptions
#df_prescriptions.shape

,Unnamed: 0,pre_id,natinal_id,sex,age,speciality,pres_gen_code,pres_full_name,number_pres_drugs,IHIO_cost,total_cost,age_days,pre_month,drug_count
0,2,2,2,0,1340/03/14,Cardiology,81680,- ANTIAGING CO Q10 RED® SOFTGEL 60 PACKAGE- 60...,60.0,0,60,22218,1,10
1,118054,2,2,0,1340/03/14,Cardiology,6788,ROSUVASTATIN TABLET ORAL 20 MG,100.0,0,630000,22218,1,10
2,70708,2,2,0,1340/03/14,Cardiology,551,FLUOXETINE CAPSULE ORAL 20 MG,60.0,46200,66000,22218,1,10
3,144595,2,2,0,1340/03/14,Cardiology,2475,CARVEDILOL TABLET ORAL 6.25 MG,200.0,168000,240000,22218,1,10
4,72312,2,2,0,1340/03/14,Cardiology,571,FUROSEMIDE TABLET ORAL 40 MG,100.0,63000,90000,22218,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11499296,16420784,3093456,1095068,1,1331/03/27,Emergency medicine,10427,VITAMIN B1/VITAMIN B6/VITAMIN B12 INJECTION PA...,4.0,0,0,25842,12,3
11499297,16377995,3093456,1095068,1,1331/03/27,Emergency medicine,1814,VITAMIN D3 INJECTION PARENTERAL 300000 [IU]/1ML,1.0,59587,78818,25842,12,3
11499298,16399979,3093456,1095068,1,1331/03/27,Emergency medicine,81201,- OSTEOQUINONE F.C TBLET 5*20,30.0,0,0,25842,12,3
11499299,16405632,3093477,581567,0,1345/08/08,Other,153,"BETAMETHASONE INJECTION, SUSPENSION, EXTENDED ...",1.0,0,0,20593,12,2


In [ ]:
# Load the first dataset (drug interactions)
df_interactions = pd.read_csv('/content/drive/My Drive/Drug Interaction/Drug Interactions for 100 most prescribed drugs/Farbod - Final/Ferrous Sulfate.csv')
df_interactions.shape

(62, 5)

In [ ]:
df_interactions = df_interactions.map(lambda x: x.strip() if isinstance(x, str) else x)
df_interactions = df_interactions.drop_duplicates(subset=['First Drug Name', 'Second Drug Name'])
df_interactions

,First Drug Name,Second Drug Name,Severity,Documentation,Summary
0,FERROUS SULFATE,ELVITEGRAVIR,Major,Excellent,Concurrent use of ELVITEGRAVIR and MEDICATIONS...
1,FERROUS SULFATE,DOLUTEGRAVIR,Major,Good,Concurrent use of DOLUTEGRAVIR and IRON may re...
2,FERROUS SULFATE,ELTROMBOPAG,Major,Good,Concurrent use of ELTROMBOPAG and IRON may res...
3,FERROUS SULFATE,CARBIDOPA,Major,Good,Concurrent use of CARBIDOPA/LEVODOPA and IRON ...
4,FERROUS SULFATE,LEVODOPA,Major,Good,Concurrent use of CARBIDOPA/LEVODOPA and IRON ...
...,...,...,...,...,...
57,FERROUS SULFATE,RABEPRAZOLE,Moderate,Fair,Concurrent use of IRON and RABEPRAZOLE may res...
58,FERROUS SULFATE,SPARFLOXACIN,Moderate,Fair,Concurrent use of IRON and SPARFLOXACIN may re...
59,FERROUS SULFATE,PANTOPRAZOLE,Moderate,Fair,Concurrent use of IRON and PANTOPRAZOLE may re...
60,FERROUS SULFATE,ACETOHYDROXAMIC ACID,Moderate,Fair,Concurrent use of ACETOHYDROXAMIC ACID and IRO...


In [ ]:
df_interactions.shape

(527, 5)

In [ ]:
df_prescriptions['natinal_id'].unique().shape[0]

982102

In [ ]:
#reduce dataset size
df_prescriptions['pre_id'] = df_prescriptions['pre_id'].astype('int32')
df_prescriptions['sex'] = df_prescriptions['sex'].astype('int8')
df_prescriptions['speciality'] = df_prescriptions['speciality'].astype('category')

In [ ]:
# Drop missing values
df_prescriptions = df_prescriptions.dropna(subset=['pres_full_name'])
df_prescriptions.shape

(11499301, 14)

In [ ]:
# Define the target drug
target_drug = "FERROUS SULFATE"


In [ ]:
# Step 1: Identify pre_ids where any row contains Target Drug (case-insensitive, word boundary match)


#for simple drugs

#pre_ids_with_target = df_prescriptions[
    #df_prescriptions['pres_full_name'].str.contains(fr"\b{re.escape(target_drug)}\b", case=False, regex=True)
#]['pre_id'].unique()


#for combination drugs or drugs with multiple target names (like iron) (put # on the lines above and remove # from the lines below)
#alternative_drugs = [
    #r"CALCIUM(?!\s*DOBESILATE)\s*\+\s*VITAMIN\s*D",
   #r"CALCIUM(?!\s*DOBESILATE)\s*D",
   # r"CALCIUM(?!\s*DOBESILATE)\s*\+\s*VIT\s*D3",
    #r"CALCIUM(?!\s*DOBESILATE)\s*\+\s*VIT\s*D"
#]

#pattern = "|".join(alternative_drugs)
#pre_ids_with_target = df_prescriptions[
#    df_prescriptions['pres_full_name'].str.contains(pattern, case=False, regex=True)
#]['pre_id'].unique()

alternative_target_drug_1 = "IRONFORTE"
alternative_target_drug_2 = "FERFOLIC"
alternative_target_drug_3 = "FOLIRON"
alternative_target_drug_4 = "IRON"
pre_ids_with_target = df_prescriptions[
  df_prescriptions['pres_full_name'].str.contains(fr"\b{re.escape(alternative_target_drug_1)}\b|\b{re.escape(target_drug)}\b|\b{re.escape(alternative_target_drug_2)}\b|\b{re.escape(alternative_target_drug_3)}\b|\b{re.escape(alternative_target_drug_4)}\b", case=False, regex=True)
]['pre_id'].unique()


pre_ids_with_target.shape[0]

23792

In [ ]:
#check which pre_id has two same drug types(to see the result you have to delete unique from the above cell)

# for simple drugs
#df_prescriptions[df_prescriptions['pres_full_name'].str.contains(fr"\b{re.escape(target_drug)}\b", case=False, regex=True)].shape

#for combination drugs or drugs with multiple target names (like iron) (put # on the lines above and remove # from the line below)
df_prescriptions[df_prescriptions['pres_full_name'].str.contains(fr"\b{re.escape(alternative_target_drug_1)}\b|\b{re.escape(target_drug)}\b|\b{re.escape(alternative_target_drug_2)}\b|\b{re.escape(alternative_target_drug_3)}\b|\b{re.escape(alternative_target_drug_4)}\b", case=False, regex=True)].shape

In [ ]:
# Step 2: Filter df_prescriptions to include all rows for these pre_ids
df_filtered = df_prescriptions[df_prescriptions['pre_id'].isin(pre_ids_with_target)]
df_filtered_azmayeshi = df_filtered[1:10000]
df_filtered

,Unnamed: 0,pre_id,natinal_id,sex,age,speciality,pres_gen_code,pres_full_name,number_pres_drugs,IHIO_cost,total_cost,age_days,pre_month,drug_count
209,185992,39,39,1,1342/06/01,General practice,7191,"IBUPROFEN CAPSULE, LIQUID FILLED ORAL 400 MG",60.0,0,0,21401,1,2
210,39,39,39,1,1342/06/01,General practice,81908,- CALCIUM CARBONATE-MINOO®TAB 10,60.0,19740,28200,21401,1,2
211,66319,40,40,1,1350/05/21,Emergency medicine,528,FAMOTIDINE TABLET ORAL 40 MG,20.0,60900,87000,18498,1,5
212,109318,40,40,1,1350/05/21,Emergency medicine,10029,"PANTOPRAZOLE CAPSULE, DELAYED RELEASE ORAL 40 MG",30.0,0,177000,18498,1,5
213,194,40,40,1,1350/05/21,Emergency medicine,82319,- FOLIRON PACKAGING 30,30.0,0,343350,18498,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11497538,16401801,3092131,1001433,1,1318/09/01,Orthopedics,80856,-SDA SIMOTRIFLEX TABLET ORAL,30.0,0,0,30431,12,4
11497539,16401803,3092132,197810,0,1338/01/08,Orthopedics,80856,-SDA SIMOTRIFLEX TABLET ORAL,30.0,0,0,23365,12,4
11497540,16381457,3092132,197810,0,1338/01/08,Orthopedics,1293,VITAMIN B1 (THIAMINE) TABLET ORAL 300 MG,30.0,300000,390000,23365,12,4
11497541,16399716,3092132,197810,0,1338/01/08,Orthopedics,81908,- CALCIUM CARBONATE-MINOO®TAB 10,30.0,0,0,23365,12,4


In [ ]:
# Step 3: Convert df_interactions to a dictionary for fast lookup and compile regex patterns
interaction_dict = {}
for agent, details in df_interactions.set_index('Second Drug Name')[['Documentation', 'Severity']].to_dict(orient='index').items():
    interaction_dict[agent] = {
        'regex': re.compile(fr"\b{re.escape(agent)}\b", re.IGNORECASE),  # Pre-compiled regex
        'Documentation': details['Documentation'],
        'Severity': details['Severity']
    }

In [ ]:
# Step 4: Prepare lists to store results
results = []
omitted_results = []

# Define non-systemic keywords
non_systemic_keywords = re.compile(r'\b(cream|lotion|topical|ointment|drops(?!\s+oral))\b', re.IGNORECASE)

# Step 5: Process each prescription (pre_id group) with a progress bar
for pre_id, group in tqdm(df_filtered.groupby('pre_id'), desc="Processing Prescriptions", unit="pre_id"):
    prescribed_drugs = group['pres_full_name'].tolist()  # List of all drugs in this prescription

    # Check if target drug is in this prescription (single drug)
    if any(re.search(fr"\b{re.escape(target_drug)}\b", drug, re.IGNORECASE) for drug in prescribed_drugs):

    #Check if target drug is in this prescription (combination drug or multiple target drugs)
    #if any(re.search(fr"\b{re.escape(alternative_target_drug)}\b|\b{re.escape(target_drug)}\b", drug, re.IGNORECASE) for drug in prescribed_drugs):

        # Count total interactions for this pre_id
        interaction_entries = []  # Store interactions temporarily
        omitted_entries = []  # Store omitted interactions

        for agent, details in interaction_dict.items():
            if any(details['regex'].search(drug) for drug in prescribed_drugs):  # Match using pre-compiled regex

                # (single drug)
                prescribed_drug = next((drug for drug in prescribed_drugs if re.search(fr"\b{re.escape(target_drug)}\b", drug, re.IGNORECASE)), target_drug)

                # (combination drug or multiple target drugs)
                #prescribed_drug = next((drug for drug in prescribed_drugs if re.search(fr"\b{re.escape(alternative_target_drug)}\b|\b{re.escape(target_drug)}\b", drug, re.IGNORECASE)), target_drug)


                interacting_drug_full_name = next((drug for drug in prescribed_drugs if details['regex'].search(drug)), None)

                # Prepare row data
                row_data = [
                    pre_id,
                    group['natinal_id'].iloc[0],
                    group['sex'].iloc[0],
                    (group['age_days'].iloc[0] / 365).astype(int),
                    group['speciality'].iloc[0],
                    group['pre_month'].iloc[0],
                    group['drug_count'].iloc[0],
                    group.loc[group['pres_full_name'] == prescribed_drug, 'IHIO_cost'].iloc[0] + group.loc[group['pres_full_name'] == interacting_drug_full_name, 'IHIO_cost'].iloc[0],
                    group.loc[group['pres_full_name'] == prescribed_drug, 'total_cost'].iloc[0] + group.loc[group['pres_full_name'] == interacting_drug_full_name, 'total_cost'].iloc[0],
                    target_drug,  # Target drug explicitly included
                    agent,         # Interacting agent added
                    details['Documentation'],
                    details['Severity'],
                    prescribed_drug,  # The full name of the drug containing the target drug
                    interacting_drug_full_name  # Exact interacting drug full name
                ]

                # Check if it's a non-systemic drug
                if non_systemic_keywords.search(prescribed_drug) or non_systemic_keywords.search(interacting_drug_full_name):
                    omitted_entries.append(row_data + ['OMITTED'])  # Add 'OMITTED' status
                else:
                    interaction_entries.append(row_data)  # Store interaction data

        # Get total number of interactions for this `pre_id`
        total_interactions = len(interaction_entries)

        # Append results with the total interaction count for all rows
        for entry in interaction_entries:
            results.append(entry + [total_interactions])

        for entry in omitted_entries:
            omitted_results.append(entry + [total_interactions])  # Keep the same count for omitted drugs

# Convert omitted results to DataFrame
df_omitted = pd.DataFrame(omitted_results, columns=[
    'pre_id', 'natinal_id', 'sex', 'age_years', 'speciality', 'pre_month', 'drug_count',
    'IHIO_cost', 'total_cost', 'target_drug', 'interacting_agent', 'Documentation', 'Severity',
    'prescribed_drug', 'interacting_drug_full_name', 'Status', 'total_interactions'
])



Processing Prescriptions: 100%|██████████| 16541/16541 [00:58<00:00, 281.40pre_id/s]


In [ ]:
df_omitted

,pre_id,natinal_id,sex,age_years,speciality,pre_month,drug_count,IHIO_cost,total_cost,target_drug,interacting_agent,Documentation,Severity,prescribed_drug,interacting_drug_full_name,Status,total_interactions
0,269913,193772,0,62,General practice,2,4,280000,1560000,MESALAZINE,PIROXICAM,Fair,Major,"MESALAZINE CAPSULE, EXTENDED RELEASE ORAL 500 MG",PIROXICAM GEL TOPICAL 0.5 G/100G 60 G,OMITTED,0
1,412471,294177,1,65,Internal medicine,3,5,2786400,3096000,MESALAZINE,DICLOFENAC,Fair,Major,"MESALAZINE TABLET, DELAYED RELEASE ORAL 500 MG",DICLOFENAC GEL TOPICAL 1 % 60 G,OMITTED,0
2,641617,406667,0,62,General practice,4,4,168000,624000,MESALAZINE,DICLOFENAC,Fair,Major,"MESALAZINE CAPSULE, EXTENDED RELEASE ORAL 500 MG",DICLOFENAC GEL TOPICAL 1 % 60 G,OMITTED,0
3,1236073,152874,1,58,General practice,6,19,444420,534420,MESALAZINE,PIROXICAM,Fair,Major,"MESALAZINE CAPSULE, EXTENDED RELEASE ORAL 500 MG",PIROXICAM GEL TOPICAL 0.5 G/100G 60 G,OMITTED,1
4,2420248,109262,1,52,General practice,10,8,0,0,MESALAZINE,PIROXICAM,Fair,Major,MESALAZINE SUPPOSITORY RECTAL 500 MG,PIROXICAM GEL TOPICAL 0.5 G/100G 60 G,OMITTED,0
5,2543675,976659,0,64,Internal medicine,11,5,2497200,4248500,MESALAZINE,PIROXICAM,Fair,Major,"MESALAZINE TABLET, DELAYED RELEASE ORAL 500 MG",PIROXICAM GEL TOPICAL 0.5 G/100G 60 G,OMITTED,0
6,2683810,123380,0,83,General practice,11,7,1656000,6525600,MESALAZINE,PIROXICAM,Fair,Major,"MESALAZINE TABLET, DELAYED RELEASE ORAL 500 MG",PIROXICAM GEL TOPICAL 0.5 G/100G 60 G,OMITTED,0
7,2771392,480896,1,43,General practice,12,3,3472000,3962100,MESALAZINE,PIROXICAM,Fair,Major,"MESALAZINE CAPSULE, EXTENDED RELEASE ORAL 500 MG",PIROXICAM GEL TOPICAL 0.5 G/100G 60 G,OMITTED,0
8,2999820,538013,0,72,Orthopedics,12,7,620000,836500,MESALAZINE,PIROXICAM,Fair,Major,"MESALAZINE CAPSULE, EXTENDED RELEASE ORAL 500 MG",PIROXICAM GEL TOPICAL 0.5 G/100G 60 G,OMITTED,1


In [ ]:
#df_omitted.to_csv("TRIFLUOPERAZINE_ommited.csv", index = False)

In [ ]:
# Step 6: Convert results to DataFrame and batch write to CSV
df_results = pd.DataFrame(results, columns=[
    'pre_id', 'natinal_id', 'sex', 'age_years', 'speciality', 'pre_month', 'drug_count',
    'IHIO_cost', 'total_cost', 'target_drug', 'interacting_agent', 'documentation', 'severity',
    'prescribed_drug', 'interacting_drug_full_name', 'interaction_count'
])

In [ ]:
##tedad kole interaction ha
df_results = df_results.map(lambda x: x.strip() if isinstance(x, str) else x)
df_results = df_results.drop_duplicates()

In [ ]:
#only for metformin and glibenclamide or for metformin and sitagliptin

# Identify rows where prescribed_drug == interacting_drug_full_name
mask = df_results['prescribed_drug'] == df_results['interacting_drug_full_name']

# Count the number of occurrences per pre_id that should be removed
removal_counts = df_results.loc[mask, 'pre_id'].value_counts()

# Reduce interaction_count for affected pre_ids by the exact number of deletions
df_results.loc[df_results['pre_id'].isin(removal_counts.index), 'interaction_count'] -= df_results['pre_id'].map(removal_counts)

# Ensure interaction_count does not go below 0
df_results['interaction_count'] = df_results['interaction_count'].clip(lower=0)

# Remove the identified rows
df_results = df_results[~mask]

# Print summary of changes
print(f"Deleted {mask.sum()} rows and updated interaction_count for {len(removal_counts)} unique pre_ids.")

Deleted 16434 rows and updated interaction_count for 16434 unique pre_ids.


In [ ]:
#df_results

In [ ]:
#for insulin as interacting agent
# Identify rows where interacting_agent is exactly "INSULIN" (case-insensitive)
insulin_mask = df_results['interacting_agent'].str.lower() == "insulin"

# Group by (pre_id, interacting_drug_full_name) to find related interactions
grouped = df_results.groupby(['pre_id', 'interacting_drug_full_name'], group_keys=False)

# Identify rows to delete
to_delete = set()
removal_counts = {}

for (pre_id, drug_name), group in grouped:
    agents = group['interacting_agent'].str.lower().unique()  # Case-insensitive check

    # If "insulin" exists but more specific insulin types are present, mark for deletion
    if "insulin" in agents and len(agents) > 1:
        rows_to_remove = group[group['interacting_agent'].str.lower() == "insulin"].index
        to_delete.update(rows_to_remove)

        # Count the number of deletions per pre_id
        removal_counts[pre_id] = removal_counts.get(pre_id, 0) + len(rows_to_remove)

# Reduce interaction_count by the exact number of deleted rows per pre_id
for pre_id, count in removal_counts.items():
    df_results.loc[df_results['pre_id'] == pre_id, 'interaction_count'] -= count

# Ensure interaction_count does not go below 0
df_results['interaction_count'] = df_results['interaction_count'].clip(lower=0)

# Remove the identified rows
df_results = df_results.drop(index=to_delete)

# Print summary of deletions
print(f"Deleted {len(to_delete)} rows and updated interaction_count for {len(removal_counts)} unique pre_ids.")

Deleted 0 rows and updated interaction_count for 0 unique pre_ids.


In [ ]:
df_results

,pre_id,natinal_id,sex,age_years,speciality,pre_month,drug_count,IHIO_cost,total_cost,target_drug,interacting_agent,documentation,severity,prescribed_drug,interacting_drug_full_name,interaction_count
0,2,2,0,60,Cardiology,1,10,134400,585000,SPIRONOLACTONE,ACETYLSALICYLIC ACID,Good,Major,SPIRONOLACTONE TABLET ORAL 25 MG,"ASA (ACETYLSALICYLIC ACID) TABLET, DELAYED REL...",2
1,2,2,0,60,Cardiology,1,10,50400,360042,SPIRONOLACTONE,VALSARTAN,Fair,Moderate,SPIRONOLACTONE TABLET ORAL 25 MG,-SACUBITRIL / VALSARTAN TABLET ORAL 97 MG/103 MG,2
2,56,56,0,44,Internal medicine,1,12,176400,252000,SPIRONOLACTONE,ACETYLSALICYLIC ACID,Good,Major,SPIRONOLACTONE TABLET ORAL 25 MG,"ASA (ACETYLSALICYLIC ACID) TABLET, DELAYED REL...",2
3,56,56,0,44,Internal medicine,1,12,343700,491000,SPIRONOLACTONE,LOSARTAN,Fair,Moderate,SPIRONOLACTONE TABLET ORAL 25 MG,LOSARTAN POTASSIUM TABLET ORAL 25 MG,2
4,99,98,0,12,Pediatrics,1,10,266000,380000,SPIRONOLACTONE,CAPTOPRIL,Good,Major,SPIRONOLACTONE TABLET ORAL 25 MG,CAPTOPRIL TABLET ORAL 25 MG,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46700,3089963,953282,0,65,General practice,12,3,8433428,8587380,SPIRONOLACTONE,INSULIN,Fair,Moderate,SPIRONOLACTONE TABLET ORAL 25 MG,"INSULIN ASPART INJECTION-MIX, SUSPENSION PAREN...",2
46701,3089963,953282,0,65,General practice,12,3,842310,997020,SPIRONOLACTONE,LOSARTAN,Fair,Moderate,SPIRONOLACTONE TABLET ORAL 25 MG,LOSARTAN POTASSIUM TABLET ORAL 50 MG,2
46702,3091560,83763,1,72,General practice,12,4,1297300,1474300,SPIRONOLACTONE,METFORMIN,Fair,Moderate,SPIRONOLACTONE TABLET ORAL 25 MG,METFORMIN HYDROCHLORIDE TABLET ORAL 500 MG,1
46703,3091688,250971,1,45,General practice,12,4,2549200,2817700,SPIRONOLACTONE,METFORMIN,Fair,Moderate,SPIRONOLACTONE TABLET ORAL 100 MG,METFORMIN HYDROCHLORIDE TABLET ORAL 500 MG,1


In [ ]:
#save to drive

safe_target_drug = target_drug.replace("/","_")

save_path = f"/content/drive/My Drive/Drug Interaction/project_results/processed_CSVs/{safe_target_drug}.csv"

# Load your DataFrame (replace 'your_dataframe' with the actual DataFrame)
df_results.to_csv(save_path, index=False)

print(f"File saved as: {save_path}")

File saved as: /content/drive/My Drive/Drug Interaction/project_results/processed_CSVs/CALCIUM CARBONATE.csv


In [ ]:
#df_prescriptions[df_prescriptions['pre_id'] == 105207]

In [ ]:
#df_results['pre_id'].unique().shape[0]

In [ ]:
#if the wrongly omitted topical drugs were added manually, remove the # and use df_results from the correct CSV.
#df_results = pd.read_csv('/content/drive/My Drive/Drug Interaction/project_results/processed_CSVs/TRIFLUOPERAZINE.csv')

In [ ]:
df_results_Contraindicated = df_results[df_results['severity'] == 'Contraindicated']
df_results_Contraindicated['pre_id'].unique().shape[0]

0

In [ ]:
df_results_Major = df_results[df_results['severity'] == 'Major']
df_results_Major['pre_id'].unique().shape[0]

638

In [ ]:
df_results_Moderate = df_results[df_results['severity'] == 'Moderate']
df_results_Moderate['pre_id'].unique().shape[0]

4038

In [ ]:
# Group by 'pre_id' and aggregate interaction types into sets
interaction_groups = df_results.groupby('pre_id')['severity'].agg(set).reset_index()

# Create a new column for the sorted, unique combinations of interaction types
interaction_groups['interaction_combination'] = interaction_groups['severity'].apply(
    lambda x: ', '.join(sorted(x))
)

# Count the unique combinations
combination_counts = interaction_groups['interaction_combination'].value_counts()

# Print all unique combinations to check if they are in the expected order
print("All unique combinations:\n", combination_counts)

# Define the exact order required, ensuring that all possible combinations are included
desired_order = [
    "Major",
    "Moderate",
    "Moderate, Major",
    "Contraindicated",
    "Major, Contraindicated",
    "Moderate, Major, Contraindicated",
    "Moderate, Contraindicated"
]

# Manually fix order to match the required format
combination_counts.index = combination_counts.index.str.replace("Major, Moderate", "Moderate, Major")
combination_counts.index = combination_counts.index.str.replace("Contraindicated, Major, Moderate", "Moderate, Major, Contraindicated")
combination_counts.index = combination_counts.index.str.replace("Contraindicated, Moderate", "Moderate, Contraindicated")
combination_counts.index = combination_counts.index.str.replace("Contraindicated, Major", "Major, Contraindicated")

# Reindex to ensure the correct order and include missing categories with count 0
combination_counts = combination_counts.reindex(desired_order, fill_value=0)

# Display the final counts
print("\nReindexed combination counts:\n", combination_counts)

All unique combinations:
 interaction_combination
Moderate           3887
Major               487
Major, Moderate     151
Name: count, dtype: int64

Reindexed combination counts:
 interaction_combination
Major                                487
Moderate                            3887
Moderate, Major                      151
Contraindicated                        0
Major, Contraindicated                 0
Moderate, Major, Contraindicated       0
Moderate, Contraindicated              0
Name: count, dtype: int64


In [ ]:
#df_results = pd.read_csv('/content/drive/My Drive/Drug Interaction/project_results/processed_CSVs/VITAMIN D.csv')
#target_drug = 'VITAMIN D'

In [ ]:
# Filter the data based on severity
moderate = df_results[df_results['severity'] == 'Moderate']
major = df_results[df_results['severity'] == 'Major']
contraindicated = df_results[df_results['severity'] == 'Contraindicated']

# Count the repetitions and unique pre_id for 'major' and 'moderate' interacting drugs
moderate_counts = moderate.groupby('interacting_agent').agg(
    repetitions=('pre_id', 'count'),  # Count the total repetitions
    unique_pre_ids=('pre_id', 'nunique')  # Count the unique pre_id
).reset_index()

major_counts = major.groupby('interacting_agent').agg(
    repetitions=('pre_id', 'count'),  # Count the total repetitions
    unique_pre_ids=('pre_id', 'nunique')  # Count the unique pre_id
).reset_index()

# Count the repetitions and unique pre_id for all contraindicated interacting drugs
contraindicated_counts = contraindicated.groupby('interacting_agent').agg(
    repetitions=('pre_id', 'count'),
    unique_pre_ids=('pre_id', 'nunique')
).reset_index()


# Get the top 3 most prevalent interacting agents in 'major' and 'moderate'
top_moderate = moderate_counts.sort_values(by='repetitions', ascending=False).head(10)
#top_moderate['target_drug'] = target_drug
# Define the path to the results file in Google Drive
#results_file_path_moderate = "/content/drive/My Drive/Drug Interaction/project_results/stat_csv/moderate.csv"

# Ensure the directory exists
#os.makedirs(os.path.dirname(results_file_path_moderate), exist_ok=True)

# Check if the file exists and is not empty
#if os.path.exists(results_file_path_moderate) and os.path.getsize(results_file_path_moderate) > 0:
 #   try:
  #      df_existing = pd.read_csv(results_file_path_moderate)
   # except pd.errors.EmptyDataError:  # Handle empty file issue
    #    df_existing = pd.DataFrame()
#else:
 #   df_existing = pd.DataFrame()

# Append the new results
#df_combined_moderate = pd.concat([df_existing, top_moderate], ignore_index=True)

# Save the updated file
#df_combined_moderate.to_csv(results_file_path_moderate, index=False)



top_major = major_counts.sort_values(by='repetitions', ascending=False).head(10)
#top_major['target_drug'] = target_drug
# Define the path to the results file in Google Drive
#results_file_path_major = "/content/drive/My Drive/Drug Interaction/project_results/stat_csv/major.csv"

# Ensure the directory exists
#os.makedirs(os.path.dirname(results_file_path_major), exist_ok=True)

# Check if the file exists and is not empty
#if os.path.exists(results_file_path_major) and os.path.getsize(results_file_path_major) > 0:
 #   try:
  #      df_existing = pd.read_csv(results_file_path_major)
   # except pd.errors.EmptyDataError:  # Handle empty file issue
    #    df_existing = pd.DataFrame()
#else:
   # df_existing = pd.DataFrame()

# Append the new results
#df_combined_major = pd.concat([df_existing, top_major], ignore_index=True)

# Save the updated file
#df_combined_major.to_csv(results_file_path_major, index=False)
# Get all contraindicated interacting agents
contraindicated_agents = contraindicated_counts[['interacting_agent', 'repetitions', 'unique_pre_ids']]

# Get the top 5 most prevalent interacting agents overall
all_counts = df_results.groupby('interacting_agent').agg(
    repetitions=('pre_id', 'count'),
    unique_pre_ids=('pre_id', 'nunique'),
).reset_index()

top_5_all = all_counts.sort_values(by='repetitions', ascending=False).head(20)
top_5_all = top_5_all.merge(
    df_results[['interacting_agent', 'severity']].drop_duplicates(),
    on='interacting_agent',
    how='left'
)
#top_5_all['target_drug'] = target_drug
# Define the path to the results file in Google Drive
#results_file_path_all = "/content/drive/My Drive/Drug Interaction/project_results/stat_csv/all.csv"

# Ensure the directory exists
#os.makedirs(os.path.dirname(results_file_path_all), exist_ok=True)

# Check if the file exists and is not empty
#if os.path.exists(results_file_path_all) and os.path.getsize(results_file_path_all) > 0:
 #   try:
  #      df_existing = pd.read_csv(results_file_path_all)
   # except pd.errors.EmptyDataError:  # Handle empty file issue
    #    df_existing = pd.DataFrame()
#else:
 #   df_existing = pd.DataFrame()

# Append the new results
#df_combined_all = pd.concat([df_existing, top_5_all], ignore_index=True)

# Save the updated file
#df_combined_all.to_csv(results_file_path_all, index=False)

# Convert results to DataFrame
#df_results_statistics = pd.DataFrame([results_statistics], columns=column_names)

# Display results
print("Top 3 Interacting Agents for 'Moderate' and 'Major':")
print(top_moderate)
print(top_major)


#print("\nAll Contraindicated Interacting Agents (with repetitions and unique pre_id):")
#print(contraindicated_agents)

print("\nTop 5 Most Prevalent Interacting Agents Overall:")
print(top_5_all)

Top 3 Interacting Agents for 'Moderate' and 'Major':
  interacting_agent  repetitions  unique_pre_ids target_drug
2        QUETIAPINE           37              37   VITAMIN D
0      ARIPIPRAZOLE            7               7   VITAMIN D
1        CIMETIDINE            2               2   VITAMIN D
Empty DataFrame
Columns: [interacting_agent, repetitions, unique_pre_ids, target_drug]
Index: []

Top 5 Most Prevalent Interacting Agents Overall:
  interacting_agent  repetitions  unique_pre_ids  severity target_drug
0        QUETIAPINE           37              37  Moderate   VITAMIN D
1      ARIPIPRAZOLE            7               7  Moderate   VITAMIN D
2        CIMETIDINE            2               2  Moderate   VITAMIN D


In [ ]:
# Define the total reference value for percentage calculations (single drug)
total_target_drugs = df_prescriptions[df_prescriptions['pres_full_name'].str.contains(
   fr"\b{re.escape(target_drug)}\b", case=False, regex=True)].shape[0]

# Define the total reference value for percentage calculations (combination drugs or multiple target drugs)
#total_target_drugs = df_prescriptions[df_prescriptions['pres_full_name'].str.contains(
  # fr"\b{re.escape(alternative_target_drug)}\b|\b{re.escape(target_drug)}\b", case=False, regex=True)].shape[0]

# Function to format counts with percentages based on a defined total
def format_counts_with_percentage(counts, total):
    return [f"{count} ({(count / total) * 100:.2f}%)" if total > 0 else f"{count} (0.00%)" for count in counts]

# Convert values to lists with percentages using total_target_drugs
contraindicated_agents_list = [
    f"{agent}: {count} ({(count / total_target_drugs) * 100:.2f}%)"
    for agent, count in zip(contraindicated_agents['interacting_agent'].str.strip(), contraindicated_agents['repetitions'])
]

top_moderate_list = [
    f"{agent}: {count} ({(count / total_target_drugs) * 100:.2f}%)"
    for agent, count in zip(top_moderate['interacting_agent'].str.strip(), top_moderate['repetitions'])
]

top_major_list = [
    f"{agent}: {count} ({(count / total_target_drugs) * 100:.2f}%)"
    for agent, count in zip(top_major['interacting_agent'].str.strip(), top_major['repetitions'])
]

top_5_all_list = [
    f"{agent}: {count} ({(count / total_target_drugs) * 100:.2f}%) - Severity: {severity}"
    for agent, count, severity in zip(
        top_5_all['interacting_agent'].str.strip(),
        top_5_all['repetitions'],
        top_5_all['severity']  # Assuming a severity column exists
    )
]

results_statistics = [
    target_drug,
    df_prescriptions.shape[0],  # Number of all drugs,
    df_prescriptions['pre_id'].unique().shape[0], # Number of all Prescriptions,
    df_interactions.shape[0],  # Number of drug interaction pairs,
    f"{pre_ids_with_target.shape[0]} ({(pre_ids_with_target.shape[0] / df_prescriptions['pre_id'].unique().shape[0]) * 100:.2f}%)",  # Absolute and percentage for prescriptions containing target drug
    f"{total_target_drugs} ({(total_target_drugs / df_prescriptions.shape[0]) * 100:.2f}%)",  # Absolute and percentage for target drugs
    f"{df_filtered.shape[0]} ({(df_filtered.shape[0] / df_prescriptions.shape[0]) * 100:.2f}%)",  # Absolute and percentage for all drugs in prescriptions containing target drug
    round((df_filtered.shape[0] / pre_ids_with_target.shape[0]), 2),
    df_results.shape[0],  # Number of all interactions,
    f"{df_results['interacting_agent'].unique().shape[0]} ({(df_results['interacting_agent'].unique().shape[0] / df_interactions.shape[0]) * 100:.2f}%)",  # Absolute and percentage for paired drug interactions found
    f"{df_results_Contraindicated.shape[0]} ({(df_results_Contraindicated.shape[0] / df_results.shape[0]) * 100:.2f}%)",  # Absolute and percentage for contraindications
    f"{df_results_Major.shape[0]} ({(df_results_Major.shape[0] / df_results.shape[0]) * 100:.2f}%)",  # Absolute and percentage for major interactions
    f"{df_results_Moderate.shape[0]} ({(df_results_Moderate.shape[0] / df_results.shape[0]) * 100:.2f}%)",  # Absolute and percentage for moderate interactions
    f"{combination_counts.values.sum()} ({(combination_counts.values.sum() / pre_ids_with_target.shape[0]) * 100:.2f}%)",  # Absolute and percentage for all interactions
    f"{df_results_Contraindicated['pre_id'].unique().shape[0]} ({(df_results_Contraindicated['pre_id'].unique().shape[0] / pre_ids_with_target.shape[0]) * 100:.2f}%)",  # Absolute and percentage for prescriptions with contraindications
    f"{df_results_Major['pre_id'].unique().shape[0]} ({(df_results_Major['pre_id'].unique().shape[0] / pre_ids_with_target.shape[0]) * 100:.2f}%)",  # Absolute and percentage for prescriptions with major interactions
    f"{df_results_Moderate['pre_id'].unique().shape[0]} ({(df_results_Moderate['pre_id'].unique().shape[0] / pre_ids_with_target.shape[0]) * 100:.2f}%)",  # Absolute and percentage for prescriptions with moderate interactions

    f"{combination_counts.values[0]} ({(combination_counts.values[0] / pre_ids_with_target.shape[0]) * 100:.2f}%)",  # Absolute and percentage for prescriptions with only major interactions
    f"{combination_counts.values[1]} ({(combination_counts.values[1] / pre_ids_with_target.shape[0]) * 100:.2f}%)",  # Absolute and percentage for prescriptions with only moderate interactions
    f"{combination_counts.values[2]} ({(combination_counts.values[2] / pre_ids_with_target.shape[0]) * 100:.2f}%)",  # Absolute and percentage for prescriptions with both major and moderate interactions
    f"{combination_counts.values[3]} ({(combination_counts.values[3] / pre_ids_with_target.shape[0]) * 100:.2f}%)",  # Absolute and percentage for prescriptions with only contraindications
    f"{combination_counts.values[4]} ({(combination_counts.values[4] / pre_ids_with_target.shape[0]) * 100:.2f}%)",  # Absolute and percentage for prescriptions with major and contraindications
    f"{combination_counts.values[5]} ({(combination_counts.values[5] / pre_ids_with_target.shape[0]) * 100:.2f}%)",  # Absolute and percentage for prescriptions with major, moderate, and contraindications
    f"{combination_counts.values[6]} ({(combination_counts.values[6] / pre_ids_with_target.shape[0]) * 100:.2f}%)",  # Absolute and percentage for prescriptions with moderate and contraindications
    ', '.join(contraindicated_agents_list),  # Contraindicated agents with counts and percentages
    ', '.join(top_moderate_list),  # Top moderate agents with counts and percentages
    ', '.join(top_major_list),  # Top major agents with counts and percentages
    ', '.join(top_5_all_list)  # Top 5 all agents with counts and percentages
]
# Column names
column_names = [
    'name',
    'number of all drugs',
    'number of all Prescriptions',
    'number of drug interaction pairs',
    'absolute and percentage of prescriptions containing target drug',
    'absolute and percentage of target drugs',
    'absolute and percentage of all drugs in prescriptions containing target drug',
    'drug average count per prescripition',
    'number of all interactions',
    'absolute and percentage of paired drug interactions found',
    'number and percentage of contraindications',
    'number and percentage of major interactions',
    'number and percentage of moderate interactions',
    'number and percentage of prescriptions containing all interactions',
    'number and percentage of prescriptions containing contraindications',
    'number and percentage of prescriptions containing major indications',
    'number and percentage of prescriptions containing moderate indications',
    'number and percentage of prescriptions containing only major interactions',
    'number and percentage of prescriptions containing only moderate interactions',
    'number and percentage of prescriptions containing both major and moderate interactions',
    'number and percentage of prescriptions containing only contraindications',
    'number and percentage of prescriptions containing major interactions and contraindications',
    'number and percentage of prescriptions containing major and moderate interactions and contraindications',
    'number and percentage of prescriptions containing moderate interactions and contraindications',
    'contraindicated agents',
    'top_moderate',
    'top_major',
    'top_5_all'
]

# Convert results to DataFrame
df_results_statistics = pd.DataFrame([results_statistics], columns=column_names)

# Define the path to the results file in Google Drive
results_file_path = "/content/drive/My Drive/Drug Interaction/project_results/stat_csv/interaction_statistics_results.csv"

# Ensure the directory exists
os.makedirs(os.path.dirname(results_file_path), exist_ok=True)

# Check if the file exists and is not empty
if os.path.exists(results_file_path) and os.path.getsize(results_file_path) > 0:
    try:
        df_existing = pd.read_csv(results_file_path)
    except pd.errors.EmptyDataError:  # Handle empty file issue
        df_existing = pd.DataFrame()
else:
    df_existing = pd.DataFrame()

# Append the new results
df_combined = pd.concat([df_existing, df_results_statistics], ignore_index=True)

# Save the updated file
df_combined.to_csv(results_file_path, index=False)



print("Results saved to 'interaction_statistics_results.csv in stat_csv folder in drive'")


Results saved to 'interaction_statistics_results.csv in stat_csv folder in drive'


In [ ]:
#df_prescriptions[df_prescriptions["pre_id"] == 6455]

In [ ]:
#df_results

In [ ]:
#boro dige ...